In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = AdaBoostClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble','AdaBoostClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble','AdaBoostClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)

# default result
# f1 = 0.0078
# mcc= 0.0113
# roc= 0.5022

# Scaling choice
with adaboost the scaling has no influence ?

In [ ]:
%%script false
# scaling choice

import matplotlib.pyplot as plt
import numpy as np

predictors = fd.getPredictors(dfTrx)
parameters={}
#parameters['max_iter']=1000

modelClf = AdaBoostClassifier(random_state=42)
duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors,[], parameters, scaler=None)
scalers = fd.getScalers()
for key in scalers:
    print(key)
    scaler=scalers.get(key)
    modelClf = AdaBoostClassifier(random_state=42)
    duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors, [],parameters,scaler=scaler)

# scaler has no adding value for the result

# Hyperparameters tuning
starting point
n_estimators=50,learning_rate=1

In [ ]:
%%script false

modelClf = AdaBoostClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'n_estimators':[5,10,25,50,75,100,200,500],
    'learning_rate':[0.1,0.5,1,1.5,2]
}
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
# 26/01/2025
# {'n_estimators': 50, 'learning_rate': 1.5} scoref1 0.046875

In [ ]:
%%script false
modelClf = AdaBoostClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'n_estimators':[30,40,50,60,70],
    'learning_rate':[1,1.5,2]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
# 26/01/2025
# dic     'n_estimators':[30,40,50,60,70],    'learning_rate':[1,1.5,2]
#{'learning_rate': 1.5, 'n_estimators': 50} scoref1 0.046875



In [ ]:
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = AdaBoostClassifier(random_state=42)
parameters={'learning_rate': 1.5, 'n_estimators':50}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble','AdaBoostClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble','AdaBoostClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

# default result (26/01/2025)

# f1 score: 0.0152
# mcc score: 0.0207
# roc auc score: 0.5044

In [ ]:
files = fd.getAllFiles()

range = []
results = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    result= fd.print_scores(dfTrx['Class'], preds,'f1', True)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    results.append(result)

fd.plt_train_test(range, results)

#test result (24/01/2025)
# d1 f1= 0.0306
# d2 f1= 0.0100
# d3 f1= 0.0194
# d7 f1= 0.0392